# Wunderground scraping

Note: this is included to show how I built a scraper for Wunderground. I ended up using a different weather site (using Selenium) because there were too many issues with ads and missing data on the Wunderground site. 

In [10]:
reset -fs

In [11]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import random

In [12]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [13]:
madis_1_hr = pd.read_csv('madis_1_hr.csv')
#madis_stations = madis_1_hr[madis_1_hr['STAID']]
madis_1_hr.columns = madis_1_hr.columns.str.strip()
madis_stations = madis_1_hr['STAID'].tolist()

In [ ]:
url_base = 'http://www.wunderground.com/history/daily/{}/date/2021-04-07'

urls = [url_base.format(s) for s in madis_stations]

import httplib2
h = httplib2.Http()

for url in urls:
    resp = h.request(url, 'HEAD')
    assert int(resp[0]['status']) < 400

In [ ]:
def rendering(url):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)                                          # load the web page from the URL
    time.sleep(10)                                            # wait for the web page to load
    render = driver.page_source                              # get the page source HTML
    driver.quit()                                            # quit ChromeDriver
    return render                                            # return the page source HTML


In [ ]:
wunderground_page = rendering(search_url)
wunderground_soup = BeautifulSoup(wunderground_page, 'html.parser')
soup_container = wunderground_soup.find('lib-city-history-summary')

In [128]:
def scrape_airport_weather(station):

    # search URL that can be formatted to find the webpage for any airport on the observation date     
    url_base = 'http://www.wunderground.com/history/daily/{}/date/2021-04-07'
    
    outfile = 'scrape_2021_04_07.csv'
    
    # if the csv file does not exist, write, if it does exist, append
    if not os.path.exists(outfile):
        mode = 'w'
    else:
        mode = 'a'
        
    with open(outfile, mode) as f:
        # write column headers for each parameter into the file for later use
        if mode == 'w':
            f.write('station,'
                    'actual_high_temp,histavg_high_temp,record_high_temp,'
                    'actual_low_temp,histavg_low_temp,record_low_temp,'
                    'actual_avg_temp,histavg_avg_temp,record_avg_temp,'
                    'actual_precip,histavg_precip,record_precip,'
                    'actual_dewpoint,histavg_dewpoint,record_dewpoint,'
                    'actual_highdewp,histavg_highdewp,record_highdewp,'
                    'actual_lowdewp,histavg_lowdewp,record_lowdewp,'
                    'actual_avgdewp,histavg_avgdewp,record_avgdewp,'
                    'actual_maxwind,histavg_maxwind,record_maxwind,'
                    'actual_visib,histavg_visib,record_visib,'
                    'actual_slp,histavg_slp,record_slp,\n')
        
        for stid in station:
            
            # format the search URL for the given airport
            url = url_base.format(stid)
        
            wunderground_page = rendering(url)
            wunderground_soup = BeautifulSoup(wunderground_page, 'html.parser')
            
            soup_container = wunderground_soup.find('lib-city-history-summary')
            if soup_container is None:
                soup_container = 'N/A'
            else:
                soup_data = soup_container.find_all('tbody', class_='ng-star-inserted')
                row = []
                for i, dat in enumerate(soup_data):
                    for j, d in enumerate(dat.find_all('tr', class_='ng-star-inserted')): # loops through High Temp, Low Temp, etc.
                        for k in d.find_all('td', class_='ng-star-inserted'): # loops through Actual, Historic Avg., Record
                            tmp = k.text
                            tmp = tmp.strip('  ')
                        
                            row.append(tmp)
                        
            f.write(stid+',') # write the station ID into the file
            f.write(','.join(row[:33]))
            f.write('\n') 

        wait = 5 * random.random()
        time.sleep(wait)
        print(f'{stid}, waited {wait:0.4} sec.')
            

In [77]:
stations_df = pd.read_csv('nws_observing_stations.csv')
stations_df.head(11)

,STID,Station Name,State,TimeZone,DayTimeSaving,Latitude,Longitude,Elevation
0,TAPA,Vc Bird Intl Airport Antigua,AG,A,n,17.117,-61.783,32.808
1,TKPN,Charlestown/Newcast,AG,A,n,17.200,-62.583,55.774
2,TRPM,Blackburne/Plymouth,AG,A,n,16.750,-62.167,39.370
3,PAAK,Atka Airport,AK,K,y,52.217,-174.200,59.055
4,PAAP,Port Alexander,AK,K,y,56.260,-134.660,3.281
5,PAAQ,Palmer,AK,K,y,61.610,-149.090,232.940
6,PABA,Barter Island,AK,K,y,70.140,-143.590,39.042
7,PABE,Bethel,AK,K,y,60.780,-161.800,125.000
8,PABI,Delta Junction/Allan AAF,AK,K,y,63.970,-145.700,1268.050
9,PABL,Buckland Airport,AK,K,y,65.990,-161.120,32.808


In [72]:
station = stations_df['STID'].tolist()
url_base = 'http://www.wunderground.com/history/daily/{}/date/2021-04-07'
url = [url_base.format(s) for s in station]

In [84]:
world_stations_df = pd.read_csv('world_stations.csv', skiprows=4)

/Users/noether/miniforge3/envs/eclipse/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [85]:
usa_stations = world_stations_df[world_stations_df['country3']=='USA']
usa_stations.head()

,country3,country2,country,region,subregion,place_name,station_name,type,stn_key,status,icao,national_id,wmo,wban,ghcn,special,lat,lon,elev
31913,USA,US,United States,NaN,NaN,"Albany, Ny.","Albany, Ny.",NaN,DZsALBAN,-,NaN,NaN,72518.0,NaN,NaN,NaN,42.691944,-73.832222,95.0
31914,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99201,-,NaN,NaN,99201.0,NaN,NaN,DB/201 CMAN/41001,34.900000,-72.900000,NaN
31915,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99203,-,NaN,NaN,99203.0,NaN,NaN,DB/203 CMAN/46001,56.300000,-148.300000,3.0
31916,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99204,-,NaN,NaN,99204.0,NaN,NaN,DB/204 CMAN/42001,25.900000,-89.700000,NaN
31917,USA,US,United States,-,-,Buoy,Hotel,NaN,USaaKC7H,-,KC7H,C7H,99207.0,NaN,NaN,DB/ CMAN/44004,38.500000,-70.700000,NaN


In [88]:
icao_stations = usa_stations[usa_stations['icao'].notna()]
icao_stations.shape

(6034, 19)

In [129]:
station = stations_df['STID'].tolist()

scrape_airport_weather(station)


/var/folders/_g/0w700wks7s1f9mzvhbcn7vvc0000gn/T/ipykernel_96236/3619567707.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.288
  (Session info: chrome=97.0.4692.99)
Stacktrace:
0   chromedriver                        0x0000000104ab4280 chromedriver + 4833920
1   chromedriver                        0x0000000104a48bf8 chromedriver + 4393976
2   chromedriver                        0x000000010463ec84 chromedriver + 158852
3   chromedriver                        0x000000010462bdfc chromedriver + 81404
4   chromedriver                        0x000000010462af48 chromedriver + 77640
5   chromedriver                        0x000000010462b2d4 chromedriver + 78548
6   chromedriver                        0x0000000104637fd8 chromedriver + 131032
7   chromedriver                        0x0000000104645264 chromedriver + 184932
8   chromedriver                        0x000000010462b584 chromedriver + 79236
9   chromedriver                        0x0000000104644ff8 chromedriver + 184312
10  chromedriver                        0x0000000104698